In [359]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/Oligo_NN.RNA_DEG.csv')
df.set_index('gene', inplace=True)
df.head()

# non_zero_genes = df[df['DEG'] != 0].index

# df = df[df.index.isin(non_zero_genes)]
df
FEATURE_NAMES = ['2mo', '9mo', '18mo', '9mo-2mo', '18mo-9mo', '9mo/2mo', '18mo/9mo', 'old-young', 'old/young', 'distance']


In [360]:
gene2value = df[['DEG']]


mcg = pd.read_csv('data/Oligo_NN.aDMR_gene.csv')
mcg_feat = mcg
mcg_feat.rename(columns={'gene_name': 'gene'}, inplace=True)
mcg_feat['distance'] = (mcg_feat['gene_start'] - mcg_feat['start']).abs().astype(np.float64)
mcg_feat['old/young'] = mcg_feat['18mo'] / mcg_feat['2mo']
mcg_feat['9mo-2mo'] = mcg_feat['9mo'] - mcg_feat['2mo']
mcg_feat['18mo-9mo'] = mcg_feat['18mo'] - mcg_feat['9mo']
mcg_feat['9mo/2mo'] = mcg_feat['9mo'] / mcg_feat['2mo']
mcg_feat['18mo/9mo'] = mcg_feat['18mo'] / mcg_feat['9mo']
mcg_feat = mcg_feat[['gene', *FEATURE_NAMES]]
mcg_feat.head()

,gene,2mo,9mo,18mo,9mo-2mo,18mo-9mo,9mo/2mo,18mo/9mo,old-young,old/young,distance
0,Rgs20,0.65,0.60,0.90,-0.05,0.30,0.923077,1.500000,0.25,1.384615,151241.0
1,Sulf1,0.36,0.52,0.56,0.16,0.04,1.444444,1.076923,0.20,1.555556,121205.0
2,Sulf1,0.43,0.59,0.64,0.16,0.05,1.372093,1.084746,0.21,1.488372,170142.0
3,Eya1,0.68,0.62,0.47,-0.06,-0.15,0.911765,0.758065,-0.21,0.691176,137980.0
4,Eya1,0.61,0.37,0.45,-0.24,0.08,0.606557,1.216216,-0.16,0.737705,138254.0


In [361]:
shared_genes = set(gene2value.index) & set(mcg_feat['gene'])
len(shared_genes)

2321

In [362]:
gene2value = gene2value[gene2value.index.isin(shared_genes)]
mcg_feat = mcg_feat[mcg_feat['gene'].isin(shared_genes)]

mcg_feat.head()

,gene,2mo,9mo,18mo,9mo-2mo,18mo-9mo,9mo/2mo,18mo/9mo,old-young,old/young,distance
0,Rgs20,0.65,0.60,0.90,-0.05,0.30,0.923077,1.500000,0.25,1.384615,151241.0
1,Sulf1,0.36,0.52,0.56,0.16,0.04,1.444444,1.076923,0.20,1.555556,121205.0
2,Sulf1,0.43,0.59,0.64,0.16,0.05,1.372093,1.084746,0.21,1.488372,170142.0
3,Eya1,0.68,0.62,0.47,-0.06,-0.15,0.911765,0.758065,-0.21,0.691176,137980.0
4,Eya1,0.61,0.37,0.45,-0.24,0.08,0.606557,1.216216,-0.16,0.737705,138254.0


In [363]:
mcg_mean = mcg_feat.groupby('gene').mean()
mcg_mean.head()
# Sort mcg_mean by gene name
mcg_mean = mcg_mean.loc[gene2value.index]
mcg_mean.head()

,2mo,9mo,18mo,9mo-2mo,18mo-9mo,9mo/2mo,18mo/9mo,old-young,old/young,distance
gene,,,,,,,,,,
Rgs20,0.650,0.600000,0.90,-0.050000,0.300000,0.923077,1.500000,0.250,1.384615,151241.000000
Sulf1,0.395,0.555000,0.60,0.160000,0.045000,1.408269,1.080834,0.205,1.521964,145673.500000
Eya1,0.630,0.453333,0.45,-0.176667,-0.003333,0.711663,1.045481,-0.180,0.715183,138168.666667
Stau2,0.370,0.570000,0.52,0.200000,-0.050000,1.540541,0.912281,0.150,1.405405,23034.000000
Ube2w,0.490,0.620000,0.70,0.130000,0.080000,1.265306,1.129032,0.210,1.428571,28480.000000


In [364]:
mcg_mean.corrwith(gene2value['DEG'])

/opt/conda/envs/gene/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:2842: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


2mo          0.034833
9mo         -0.016470
18mo        -0.038960
9mo-2mo     -0.046858
18mo-9mo    -0.060399
9mo/2mo           NaN
18mo/9mo          NaN
old-young   -0.060147
old/young         NaN
distance     0.101508
dtype: float64

In [365]:
# Train a sequence model on mcg_feat to predict gene2value['log2(old/young)']
# Each gene has a sequence of 4 features, 2mo, 9mo, 18mo, old-young
# The sequence length is not fixed, so we need to use a dynamic model
# Let's use a commonly used sequence prediction model for sentence classification
# like LSTM or Transformer

# Step 1: Prepare the data
list_mcg_feat = mcg_feat.groupby('gene').apply(lambda x: x[FEATURE_NAMES].values.tolist())
x = list_mcg_feat.values.tolist()

# # Find the maximum length and number of features
# max_len = max(len(seq) for seq in x)
# n_features = len(x[0][0])

# # Pad sequences to the same length and reshape
# x_padded = np.full((len(x), max_len, n_features), 0)
# for i, seq in enumerate(x):
#     x_padded[i, :len(seq), :] = seq

# # x_padded is now a 3D numpy array with shape (n_samples, max_len, n_features)
# print(x_padded.shape)

/var/tmp/ipykernel_2659864/1269483849.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  list_mcg_feat = mcg_feat.groupby('gene').apply(lambda x: x[FEATURE_NAMES].values.tolist())


In [366]:
y = gene2value.loc[list_mcg_feat.index]['DEG'].values.tolist()
y = np.array([int(i) for i in y])
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [367]:
# Step 2: Split the data into training and testing sets
from sklearn.model_selection import train_test_split
import numpy as np

# Separate the data into zero and non-zero y values
zero_indices = np.where(y == 0)[0]
non_zero_indices = np.where(y != 0)[0]
print(f'zero: {len(zero_indices)}, non-zero: {len(non_zero_indices)}')

# Sample len(non_zero_indices) indices from each group
n_samples = len(non_zero_indices)
sampled_zero_indices = np.random.choice(zero_indices, n_samples // 2, replace=False)
sampled_non_zero_indices = np.random.choice(non_zero_indices, n_samples, replace=False)

# Combine the sampled indices
sampled_indices = np.concatenate([sampled_zero_indices, sampled_non_zero_indices])

# Create balanced dataset
X_balanced = [x[i] for i in sampled_indices]
y_balanced = y[sampled_indices]

# Split the balanced dataset into training and testing sets
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=25)


zero: 2116, non-zero: 205


In [368]:
# Normalize the features

# Normalization function
def normalize_features(train_data, test_data):
    # Flatten the lists for easier processing
    train_flat = [item for sublist in train_data for item in sublist]
    test_flat = [item for sublist in test_data for item in sublist]
    
    # Separate features
    train_other_features = np.array([item[:len(FEATURE_NAMES)-1] for item in train_flat])
    train_distances = np.array([item[len(FEATURE_NAMES)-1] for item in train_flat])
    test_other_features = np.array([item[:len(FEATURE_NAMES)-1] for item in test_flat])
    test_distances = np.array([item[len(FEATURE_NAMES)-1] for item in test_flat])
    
    # Normalize other features using min-max scaling based on train data
    min_vals = np.min(train_other_features, axis=0)
    max_vals = np.max(train_other_features, axis=0)
    train_normalized_features = (train_other_features - min_vals) / (max_vals - min_vals)
    test_normalized_features = (test_other_features - min_vals) / (max_vals - min_vals)
    
    # Normalize distances using log transformation and then min-max scaling based on train data
    train_log_distances = np.log1p(train_distances)
    test_log_distances = np.log1p(test_distances)
    min_dist = np.min(train_log_distances)
    max_dist = np.max(train_log_distances)
    train_normalized_distances = (train_log_distances - min_dist) / (max_dist - min_dist)
    test_normalized_distances = (test_log_distances - min_dist) / (max_dist - min_dist)
    
    # Combine normalized features and distances
    def reconstruct_data(features, distances, original_data):
        normalized_data = []
        idx = 0
        for sublist in original_data:
            normalized_sublist = []
            for _ in sublist:
                normalized_sublist.append(list(features[idx][:len(FEATURE_NAMES)-1]) + [distances[idx]])
                idx += 1
            normalized_data.append(normalized_sublist)
        return normalized_data
    
    train_normalized = reconstruct_data(train_normalized_features, train_normalized_distances, train_data)
    test_normalized = reconstruct_data(test_normalized_features, test_normalized_distances, test_data)
    
    return train_normalized, test_normalized

# Apply normalization
X_train_normalized, X_test_normalized = normalize_features(X_train_raw, X_test_raw)

print('train:', len(X_train_normalized), 'test:', len(X_test_normalized))


train: 245 test: 62


In [369]:
y_test_raw[:10]

array([ 0, -1,  1,  1,  0,  0,  1,  1, -1, -1])

In [370]:
# Step 3: Train a sequence model on the training set
# Use pytorch to train a LSTM classifier
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence


class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

# Initialize the model
model = LSTMClassifier(input_size=len(FEATURE_NAMES), hidden_size=16, num_classes=3)
# Initialize the weights
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

model.apply(init_weights)

# Pad sequences to the same length
def pad_sequences(sequences):
    return pad_sequence([torch.FloatTensor(seq) for seq in sequences], batch_first=True)

X_train = pad_sequences(X_train_normalized)
X_test = pad_sequences(X_test_normalized)
y_train = torch.tensor([int(i) + 1 for i in y_train_raw], dtype=torch.long)
y_test = torch.tensor([int(i) + 1 for i in y_test_raw], dtype=torch.long)


# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Train the model
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(50):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        predictions = test_outputs.argmax(dim=1)
        test_acc = (test_outputs.argmax(dim=1) == y_test).float().mean()
        print(f'Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Test Acc: {test_acc:.4f}')


Epoch 1, Train Loss: 1.1344, Test Acc: 0.3710
Epoch 2, Train Loss: 1.1059, Test Acc: 0.3548
Epoch 3, Train Loss: 1.1107, Test Acc: 0.3548
Epoch 4, Train Loss: 1.1070, Test Acc: 0.3548
Epoch 5, Train Loss: 1.0900, Test Acc: 0.3548
Epoch 6, Train Loss: 1.0916, Test Acc: 0.3548
Epoch 7, Train Loss: 1.0933, Test Acc: 0.3548
Epoch 8, Train Loss: 1.0997, Test Acc: 0.3710
Epoch 9, Train Loss: 1.0936, Test Acc: 0.3548
Epoch 10, Train Loss: 1.0962, Test Acc: 0.3548
Epoch 11, Train Loss: 1.0884, Test Acc: 0.3548
Epoch 12, Train Loss: 1.0926, Test Acc: 0.3548
Epoch 13, Train Loss: 1.0909, Test Acc: 0.3548
Epoch 14, Train Loss: 1.0991, Test Acc: 0.3548
Epoch 15, Train Loss: 1.0923, Test Acc: 0.3548
Epoch 16, Train Loss: 1.0914, Test Acc: 0.3548
Epoch 17, Train Loss: 1.0888, Test Acc: 0.3548
Epoch 18, Train Loss: 1.0935, Test Acc: 0.3548
Epoch 19, Train Loss: 1.0884, Test Acc: 0.3548
Epoch 20, Train Loss: 1.0899, Test Acc: 0.3548
Epoch 21, Train Loss: 1.1088, Test Acc: 0.3548
Epoch 22, Train Loss: 

In [371]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Set random seed
torch.manual_seed(25)

# Define the attention-based model
class AttentionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(AttentionModel, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        self.classifier = nn.Linear(input_dim, output_dim)
    
    def forward(self, x, mask):
        attention_weights = self.attention(x).squeeze(-1)
        attention_weights = attention_weights.masked_fill(mask == 0, float('-inf'))
        attention_weights = torch.softmax(attention_weights, dim=1)
        
        weighted_sum = torch.sum(x * attention_weights.unsqueeze(-1), dim=1)
        
        output = self.classifier(weighted_sum)
        return output, attention_weights

# Custom dataset
class GeneDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        gene_data = torch.FloatTensor(self.data[idx])
        label = torch.LongTensor([self.labels[idx] + 1])  # Add 1 to shift labels to 0, 1, 2
        mask = torch.ones(len(gene_data))
        return gene_data, label, mask

# Collate function for DataLoader
def collate_fn(batch):
    # Sort the batch by sequence length (descending)
    batch.sort(key=lambda x: len(x[0]), reverse=True)
    sequences, labels, masks = zip(*batch)
    
    # Get lengths of each sequence
    lengths = [len(seq) for seq in sequences]
    max_len = max(lengths)
    
    # Pad sequences
    padded_seqs = torch.zeros(len(sequences), max_len, sequences[0].size(1))
    padded_masks = torch.zeros(len(sequences), max_len)
    
    for i, (seq, length) in enumerate(zip(sequences, lengths)):
        padded_seqs[i, :length] = seq
        padded_masks[i, :length] = 1
    
    return padded_seqs, torch.cat(labels), padded_masks

# Create datasets and dataloaders
train_dataset = GeneDataset(X_train_normalized, y_train_raw)
test_dataset = GeneDataset(X_test_normalized, y_test_raw)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Initialize the model
input_dim = len(FEATURE_NAMES)  # number of features per region
hidden_dim = 64
output_dim = 3  # number of classes (-1, 0, 1)
model = AttentionModel(input_dim, hidden_dim, output_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.0001)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    train_correct = 0
    train_total = 0
    for batch_x, batch_y, batch_mask in train_loader:
        optimizer.zero_grad()
        outputs, _ = model(batch_x, batch_mask)
        loss = criterion(outputs, batch_y.squeeze())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        train_correct += (outputs.argmax(dim=1) == batch_y.squeeze()).sum().item()
        train_total += batch_y.size(0)

    # scheduler.step()
    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_x, batch_y, batch_mask in test_loader:
            outputs, _ = model(batch_x, batch_mask)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y.squeeze()).sum().item()
    
    accuracy = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_correct/train_total:.4f}, Test Accuracy: {accuracy:.4f}')

# Final evaluation
model.eval()
all_attention_weights = []
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch_x, batch_y, batch_mask in test_loader:
        outputs, attention_weights = model(batch_x, batch_mask)
        _, predicted = torch.max(outputs.data, 1)
        all_attention_weights.extend(attention_weights.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

# Print final accuracy
final_accuracy = sum(np.array(all_predictions) == np.array(all_labels).squeeze()) / len(all_labels)
print(f'Final Test Accuracy: {final_accuracy:.4f}')


Epoch [1/50], Train Loss: 1.1156, Train Accuracy: 0.3020, Test Accuracy: 0.3387
Epoch [2/50], Train Loss: 1.0943, Train Accuracy: 0.3633, Test Accuracy: 0.3548
Epoch [3/50], Train Loss: 1.0902, Train Accuracy: 0.3837, Test Accuracy: 0.3387
Epoch [4/50], Train Loss: 1.0806, Train Accuracy: 0.3837, Test Accuracy: 0.3226
Epoch [5/50], Train Loss: 1.0814, Train Accuracy: 0.3918, Test Accuracy: 0.3387
Epoch [6/50], Train Loss: 1.0742, Train Accuracy: 0.4122, Test Accuracy: 0.3871
Epoch [7/50], Train Loss: 1.0648, Train Accuracy: 0.4245, Test Accuracy: 0.3387
Epoch [8/50], Train Loss: 1.0599, Train Accuracy: 0.4531, Test Accuracy: 0.4355
Epoch [9/50], Train Loss: 1.0514, Train Accuracy: 0.4612, Test Accuracy: 0.4355
Epoch [10/50], Train Loss: 1.0516, Train Accuracy: 0.4694, Test Accuracy: 0.4032
Epoch [11/50], Train Loss: 1.0401, Train Accuracy: 0.4816, Test Accuracy: 0.4355
Epoch [12/50], Train Loss: 1.0396, Train Accuracy: 0.4653, Test Accuracy: 0.4355
Epoch [13/50], Train Loss: 1.0345, Tr